In [1]:
"""
Solves the incompressible Navier-Stokes equations in conjunction with
an advection equation in a closed 3D box, and creates an animation of the result.

Momentum:           ∂u/∂t + (u ⋅ ∇) u = − 1/ρ ∇p + ν ∇²u + f
Incompressibility:  ∇ ⋅ u = 0
Advection:          ∂s/∂t + (u ⋅ ∇) s = α ∇²s + i

u:  Velocity (3d vector)
p:  Pressure
f:  Forcing (here due to Buoyancy)
ν:  Kinematic Viscosity
ρ:  Density (here =1.0)
t:  Time
∇:  Nabla operator (defining nonlinear convection, gradient and divergence)
∇²: Laplace Operator
s:  Concentration of a species (here hot smoke)
α:  Diffusivity of the embedded species
i:  Inflow of hot smoke into the domain.
"""

from phi.jax.flow import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm
import numpy as np
import torch


N_TIME_STEPS = 300
cells = 8
size = 50 

def save_data_for_gino(velocity, smoke, pressure, step):
    # Convert to numpy arrays
    # v_np = velocity.values.numpy('y,x,z,vector')
    s_np = smoke.values.numpy('y,x,z')
    p_np = pressure.values.numpy('y,x,z')

    # Save as PyTorch tensors
    torch.save({
        # 'velocity': torch.from_numpy(v_np),
        'smoke': torch.from_numpy(s_np),
        'pressure': torch.from_numpy(p_np)
    }, f'data/simulation_data_step_{step}.pt')

def main():
    # Initialize velocity field
    velocity = StaggeredGrid(
        values=(0, 0, 0),
        extrapolation=0,
        x=cells, y=cells, z=cells,
        bounds=Box(x=size, y=size, z=size)
    )

    # Initialize smoke concentration field
    smoke = CenteredGrid(
        values=0,
        extrapolation=ZERO_GRADIENT,
        x=cells, y=cells, z=cells,
        bounds=Box(x=size, y=size, z=size)
    )

    # Define inflow of smoke
    inflow = 0.2 * resample(Sphere(x=size/2, y=size/2, z=10/2, radius=5), to=smoke, soft=True)

    # Initialize pressure field
    pressure = None

    @jit_compile
    def step(v, s, p, dt=1.):
        s = advect.mac_cormack(s, v, dt) + inflow
        buoyancy = resample(s * (0, 0, 0.1), to=v)
        v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
        v, p = fluid.make_incompressible(v, (), Solve('auto', 1e-5, x0=p))
        return v, s, p

    # Set up the figure and axis for animation
    fig, ax = plt.subplots(figsize=(8, 8))
    # plt.style.use("dark_background")

    # Initialize an empty list to store the animation frames
    frames = []

    # Run simulation and collect frames
    for i in tqdm(range(N_TIME_STEPS)):
        velocity, smoke, pressure = step(velocity, smoke, pressure)
        save_data_for_gino(velocity, smoke, pressure, i)
        
        # Extract 2D slice from the middle of the 3D volume for visualization
        smoke_volume = smoke.values.numpy('y,x,z')
        smoke_slice_yz = smoke_volume[:, int(cells/2), :]
        
        # Add frame to the list
        frames.append([ax.imshow(smoke_slice_yz, origin="lower", cmap='jet', 
                                 extent=[0, 100, 0, 100], animated=True)])

    # Animation function
    def animate(frame):
        ax.clear()
        im = ax.imshow(frame[0].get_array(), origin="lower", cmap='inferno', extent=[0, 100, 0, 100])
        ax.set_title(f"Smoke Plume Simulation (Step {frame[0].get_array().frame_num+1})")
        ax.set_xlabel("Z")
        ax.set_ylabel("Y")
        return [im]

    # Create the animation
    anim = animation.ArtistAnimation(fig, frames, interval=50, blit=True, repeat_delay=1000)

    # Set up formatting for the movie files
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

    # Save the animation
    anim.save('smoke_plume_simulation.mp4', writer=writer)

    plt.close(fig)
    print("Animation saved as 'smoke_plume_simulation.mp4'")

if __name__ == "__main__":
    main()

100%|██████████| 300/300 [00:07<00:00, 38.89it/s] 


Animation saved as 'smoke_plume_simulation.mp4'
